In [17]:
from datasets import load_dataset
import openai
import os
import time
import pandas as pd
from scipy.spatial.distance import cosine
import pickle
from tqdm import tqdm 

In [18]:
dataset = pd.read_excel('/Users/jd/trauma_llm/trauma_data/embedding_data_reorganized.xlsx')

total_response = 2

res = dataset['Value 1']
for ii in range(total_response-1):
    res += dataset['Value '+str(ii+2)]
res /= total_response

dataset['score'] = res

In [19]:
dataset.head()


,Response 1,Response 2,Value 1,Value 2,score
0,Obtain a trauma series of xrays followed by a ...,Perform primary trauma survey with attention t...,5,3,2.5
1,"Primary, secondary survey, X-ray of RLE to eva...",splint the leg and transfer them to a level 1 ...,8,4,4.0
2,Immediate transfer to trauma center. Consider ...,Stabilize and transfer,9,4,4.5
3,"1. Check VS and perform a primary survey, obta...","Primary Survey, follow ATLS, CXR/PXR w/ RLE im...",6,4,3.0
4,Transfer emergently to a Level 1 or 2 trauma c...,"I would obtain an anticoagulation history, ass...",9,4,4.5


In [12]:
# Load dataset
#dataset = load_dataset("sentence-transformers/stsb", split="train")
'''dataset = load_dataset("tabilab/biosses", split="train")
print(f"Loaded {len(dataset)} examples. Sample:")
print(dataset[0], "\n")'''


'dataset = load_dataset("tabilab/biosses", split="train")\nprint(f"Loaded {len(dataset)} examples. Sample:")\nprint(dataset[0], "\n")'

In [20]:
# Set your OpenAI API key (best via environment variable)
openai.api_key = os.getenv("OPENAI_API_KEY")

In [21]:
# Function to get embedding
def get_embedding(text, model="text-embedding-3-large"):
    response = openai.embeddings.create(
        model=model,
        input=[text]
    )
    # Extract the embedding vector
    return response.data[0].embedding

In [22]:
# Generate embeddings for the first few sentence pairs (do not run without need, save results, costs money)
file_to_save = 'trauma/openai_text-embedding-3-large.pickle'
embeddings = pd.DataFrame()
sentence1 = []
sentence2 = []
embedding1 = []
embedding2 = []
cosine_similarity = []
human_score = []

print('Extracting embedding and measuring similarity ...')
time.sleep(1)
for idx in tqdm(range(len(dataset))):
    first_sentence = dataset["Response 1"][idx]
    emb1 = get_embedding(first_sentence)
    sentence1.append(
        first_sentence
    )
    embedding1.append(
        emb1
    )
    #############################
    second_sentence = dataset["Response 2"][idx]
    emb2 = get_embedding(second_sentence)
    sentence2.append(
        second_sentence
    )
    embedding2.append(
        emb2
    )
    
    ############################
    cosine_similarity.append(
        1 - cosine(emb1, emb2)
    )
    human_score.append(
        dataset["score"][idx]/5.0
    )
    
    time.sleep(1)  # respect rate limits!

embeddings['Sentence 1'] = sentence1
embeddings['Sentence 2'] = sentence2
embeddings['Embedding of Sentence 1'] = embedding1
embeddings['Embedding of Sentence 2'] = embedding2
embeddings['Cosine similarity'] = cosine_similarity
embeddings['Human score'] = human_score

with open(file_to_save, 'wb') as f:
    pickle.dump(embeddings, f)

Extracting embedding and measuring similarity ...


100%|██████████| 234/234 [06:35<00:00,  1.69s/it]


In [23]:
embeddings.head()

,Sentence 1,Sentence 2,Embedding of Sentence 1,Embedding of Sentence 2,Cosine similarity,Human score
0,Obtain a trauma series of xrays followed by a ...,Perform primary trauma survey with attention t...,"[0.034642159938812256, 0.02389717288315296, -0...","[0.027597464621067047, 0.02002732641994953, -0...",0.651575,0.5
1,"Primary, secondary survey, X-ray of RLE to eva...",splint the leg and transfer them to a level 1 ...,"[0.0030536833219230175, 0.0001038115588016808,...","[0.021354706957936287, -0.009206251241266727, ...",0.599870,0.8
2,Immediate transfer to trauma center. Consider ...,Stabilize and transfer,"[0.006583758629858494, 0.01243332028388977, 0....","[-0.007823437452316284, -0.012772772461175919,...",0.415545,0.9
3,"1. Check VS and perform a primary survey, obta...","Primary Survey, follow ATLS, CXR/PXR w/ RLE im...","[0.025017229840159416, 0.010999794118106365, -...","[0.012811556458473206, 0.007395625114440918, -...",0.722403,0.6
4,Transfer emergently to a Level 1 or 2 trauma c...,"I would obtain an anticoagulation history, ass...","[0.007118304260075092, -0.0008773026638664305,...","[-0.007306366227567196, -0.015579333528876305,...",0.567349,0.9
